In [ ]:
%load_ext autoreload
%autoreload 2
import datajoint as dj
from pipeline import lab, get_schema_name, experiment, foraging_model

# Migrate my previous code

## Define models

In [ ]:
dj.ERD(foraging_model)

In [ ]:
foraging_model.ModelClass()

In [ ]:
foraging_model.Model.load_models()

In [ ]:
foraging_model.Model & 'is_choice_kernel'

In [ ]:
foraging_model.Model() * foraging_model.Model.Param() & 'model_id=0'

In [ ]:
foraging_model.FittedSessionModel.key_source

## Prepare data for each session

In [ ]:
dj.ERD(experiment.BehaviorTrial) +2

In [ ]:
dj.ERD(experiment.WaterPort)+1 + dj.ERD(experiment.SessionBlock) 

In [ ]:
key = {'subject_id': 447921, 'session': 3, 'model_id': 5}
experiment.WaterPortChoice.proj(choice='water_port') * experiment.BehaviorTrial.proj('outcome', 'early_lick') * \
experiment.SessionBlock.BlockTrial & key

## Populate model fitting

In [ ]:
schema = dj.schema(get_schema_name('foraging_model'))
schema.jobs

In [ ]:
schema.jobs.fetch('key')

In [ ]:
finished = len(foraging_model.FittedSessionModel())
total = len(foraging_model.FittedSessionModel.key_source)
print(f'Fitted session: {finished}/{total}, {finished/total:.2%}')

In [ ]:
foraging_model.FittedSessionModel()